## 粘度预测模型（Viscosity Prediction）

本代码实现了一个**基于离子对 SMILES 结构的离子液体粘度预测模型**，整体设计严格参考论文  
**《Predicting Ionic Liquid Materials Properties from Chemical Structure》**，用于复现实验并支持后续迁移学习。

---

### 一、模型整体思路

模型以**阳离子 / 阴离子的分子图结构**为输入，使用 **Message Passing Neural Network (MPNN)** 分别对两种离子进行编码，再将其特征进行融合，最终在**物理约束条件下**预测粘度的对数值。

---

### 二、核心结构

- **输入**
  - 阳离子与阴离子的：
    - 原子类别（atom ids）
    - 键类型（bond ids）
    - 图连接关系（edge indices）
  - 温度 `T`（显式作为模型输入）

- **分子编码（MPNN）**
  - 原子 / 键嵌入（Embedding）
  - 多步消息传递（BondMatrixMessage + GatedUpdate）
  - 全局池化（GlobalSumPool）得到分子指纹

- **离子对融合**
  - 对阳离子与阴离子指纹进行逐元素相加（paper-consistent）
  - 经全连接层生成粘度物理参数

---

### 三、物理约束粘度头

模型并非直接回归粘度，而是学习物理参数：

\[
\log(\eta) = A + \frac{B}{T + C}
\]

其中：
- \(A\)：无约束偏置项  
- \(B\)：通过 `softplus + clip` 约束为正值  
- \(C\)：保证温度平移项为合理正区间  
- 温度 \(T\) 进行物理尺度归一化  

该设计确保预测结果**符合实际物理规律**。

---

### 四、训练与评估

- **数据划分**
  - 默认采用论文中的随机划分（存在 pair-level leakage）
  - 代码中预留了 **严格无泄漏划分（按离子对）** 的可选方案

- **训练设置**
  - Optimizer：Adam（带梯度裁剪）
  - Loss：MSE
  - Early Stopping 防止过拟合
  - 自定义 Callback 控制关键 epoch 输出

- **评估指标**
  - R²（决定系数）
  - MAE（平均绝对误差）

---

### 五、结果可视化与保存

- 绘制训练 / 验证损失曲线
- 生成论文对应的 **Figure 2(a)**：实验值 vs 预测值散点图
- 保存最终模型为 `.keras` 格式，用于后续迁移学习任务

---



In [1]:
# ============================================
# Cell 1: Environment setup & global config
# - Suppress TF logs
# - Import all core dependencies
# ============================================

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

from tensorflow.keras.layers import Input, Embedding, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import train_test_split

from models.layers import (
    BondMatrixMessage,
    GatedUpdate,
    GlobalSumPool,
    Reduce
)

EPS = 1e-6


In [2]:
# ============================================
# Cell 2: Utility functions
# - R2 metric
# - Padding helpers
# - Training curve plotting
# ============================================

def r2_numpy(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1.0 - ss_res / (ss_tot + EPS)


def pad_sequences_1d(seq_list, max_len, pad_val=0):
    return np.array(
        [s + [pad_val] * (max_len - len(s)) for s in seq_list],
        dtype=np.int32
    )


def plot_loss(history, out_path="loss_curve_viscosity.png"):
    plt.figure(figsize=(6, 4))
    plt.plot(history.history["loss"], label="Train loss")
    plt.plot(history.history["val_loss"], label="Validation loss")
    plt.xlabel("Epoch")
    plt.ylabel("MSE loss")
    plt.title("Training curve (viscosity)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path, dpi=300)
    plt.close()


In [3]:
# ============================================
# Cell 3: Graph preprocessing & custom callback
# - Edge duplication (undirected graph)
# - Selective verbose logging
# ============================================

def preprocess_edges_and_bonds(edge_list, bond_list, max_edges):
    processed_edges, processed_bonds = [], []

    for edges, bonds in zip(edge_list, bond_list):
        e2, b2 = [], []
        for (src, tgt), bond_id in zip(edges, bonds):
            e2.append([src, tgt])
            b2.append(bond_id)
            e2.append([tgt, src])
            b2.append(bond_id)
        processed_edges.append(e2)
        processed_bonds.append(b2)

    max_len = max_edges * 2

    processed_edges = [
        e + [[0, 0]] * (max_len - len(e)) if len(e) < max_len else e[:max_len]
        for e in processed_edges
    ]
    processed_bonds = [
        b + [0] * (max_len - len(b)) if len(b) < max_len else b[:max_len]
        for b in processed_bonds
    ]

    return (
        np.array(processed_edges, dtype=np.int32),
        np.array(processed_bonds, dtype=np.int32)
    )


class SelectiveVerboseCallback(keras.callbacks.Callback):
    def __init__(self, total_epochs):
        super().__init__()
        base = [1, 2, 3, 4, 5, 50, 100, 150, 200]
        last_five = list(range(total_epochs - 4, total_epochs + 1))
        self.verbose_epochs = set(base + last_five)

    def on_epoch_end(self, epoch, logs=None):
        ep = epoch + 1
        if ep in self.verbose_epochs:
            print(
                f"Epoch {ep} - "
                f"loss: {logs['loss']:.6f}, "
                f"val_loss: {logs['val_loss']:.6f}"
            )


In [4]:
# ============================================
# Cell 4: Physically constrained viscosity head
# log(η) = A + B / (T + C)
# ============================================

@keras.saving.register_keras_serializable(package="ionic_mpnn")
def get_A(x):
    return keras.ops.expand_dims(x[:, 0], -1)


@keras.saving.register_keras_serializable(package="ionic_mpnn")
def get_B(x):
    return keras.ops.clip(
        keras.ops.nn.softplus(keras.ops.expand_dims(x[:, 1], -1)),
        0.0, 20.0
    )


@keras.saving.register_keras_serializable(package="ionic_mpnn")
def get_C(x):
    return keras.ops.clip(
        keras.ops.nn.softplus(keras.ops.expand_dims(x[:, 2], -1)),
        0.1, 50.0
    )


@keras.saving.register_keras_serializable(package="ionic_mpnn")
def compute_log_eta(inputs):
    A, B, T, C = inputs
    return A + B / (T + C + 1e-3)


In [5]:
# ============================================
# Cell 5: MPNN model definition (paper settings)
# ============================================

def build_model(
    atom_vocab_size,
    bond_vocab_size,
    atom_dim=32,
    bond_dim=8,
    fp_size=32,
    mixing_size=20,
    num_steps=4
):
    cat_atom = Input(shape=(None,), dtype=tf.int32, name="cat_atom")
    cat_bond = Input(shape=(None,), dtype=tf.int32, name="cat_bond")
    cat_conn = Input(shape=(None, 2), dtype=tf.int32, name="cat_connectivity")

    an_atom = Input(shape=(None,), dtype=tf.int32, name="an_atom")
    an_bond = Input(shape=(None,), dtype=tf.int32, name="an_bond")
    an_conn = Input(shape=(None, 2), dtype=tf.int32, name="an_connectivity")

    T_input = Input(shape=(1,), dtype=tf.float32, name="temperature")

    atom_emb = Embedding(atom_vocab_size, atom_dim)
    bond_emb = Embedding(bond_vocab_size, bond_dim)

    def encode(atom_ids, bond_ids, conn, prefix):
        h = atom_emb(atom_ids)
        b = bond_emb(bond_ids)

        for i in range(num_steps):
            m = BondMatrixMessage(atom_dim, bond_dim)([h, b, conn])
            agg = Reduce()([m, conn[:, :, 1], h])
            h = GatedUpdate(atom_dim)([h, agg])

        fp = GlobalSumPool()([h, atom_ids])
        fp = Dense(fp_size, activation="relu", kernel_regularizer=l2(1e-5))(fp)
        return fp

    fp_cat = encode(cat_atom, cat_bond, cat_conn, "cat")
    fp_an  = encode(an_atom,  an_bond,  an_conn,  "an")

    mixed = Dense(mixing_size, activation="relu")(fp_cat + fp_an)

    params = Dense(3)(mixed)
    A = Lambda(get_A)(params)
    B = Lambda(get_B)(params)
    C = Lambda(get_C)(params)

    T_scaled = Lambda(lambda t: t / 100.0)(T_input)
    log_eta = Lambda(compute_log_eta)([A, B, T_scaled, C])

    model = Model(
        inputs=[
            cat_atom, cat_bond, cat_conn,
            an_atom, an_bond, an_conn,
            T_input
        ],
        outputs=log_eta
    )

    model.compile(
        optimizer=keras.optimizers.Adam(1e-3, clipnorm=1.0),
        loss="mse"
    )
    return model


In [6]:
# ============================================
# Cell 6: Data loading, training, evaluation
# ============================================

DATA = "data/viscosity_id_data.pkl"
VOCAB = "data/vocab.pkl"

with open(DATA, "rb") as f:
    data = pickle.load(f)
with open(VOCAB, "rb") as f:
    vocab = pickle.load(f)

atom_vocab_size = vocab["atom_vocab_size"] + 1
bond_vocab_size = vocab["bond_vocab_size"] + 1

pair_ids = [d["pair_id"] for d in data]

cat_atoms = [[a + 1 for a in d["cation"]["atom_ids"]] for d in data]
cat_bonds = [[b + 1 for b in d["cation"]["bond_ids"]] for d in data]
cat_edges = [d["cation"]["edge_indices"] for d in data]

an_atoms  = [[a + 1 for a in d["anion"]["atom_ids"]] for d in data]
an_bonds  = [[b + 1 for b in d["anion"]["bond_ids"]] for d in data]
an_edges  = [d["anion"]["edge_indices"] for d in data]

T = np.array([d["T"] for d in data], np.float32)[:, None]
y = np.array([d["log_eta"] for d in data], np.float32)

idx = np.arange(len(data))
idx_train, idx_tmp = train_test_split(idx, test_size=0.30, random_state=42)
idx_dev, idx_test  = train_test_split(idx_tmp, test_size=0.50, random_state=42)

max_atoms = max(max(map(len, cat_atoms)), max(map(len, an_atoms)))
max_edges = max(max(map(len, cat_edges)), max(map(len, an_edges)))

def build_inputs(idxs):
    ce, cb = preprocess_edges_and_bonds(
        [cat_edges[i] for i in idxs],
        [cat_bonds[i] for i in idxs],
        max_edges
    )
    ae, ab = preprocess_edges_and_bonds(
        [an_edges[i] for i in idxs],
        [an_bonds[i] for i in idxs],
        max_edges
    )
    return {
        "cat_atom": pad_sequences_1d([cat_atoms[i] for i in idxs], max_atoms),
        "cat_bond": cb,
        "cat_connectivity": ce,
        "an_atom": pad_sequences_1d([an_atoms[i] for i in idxs], max_atoms),
        "an_bond": ab,
        "an_connectivity": ae,
        "temperature": T[idxs]
    }

x_train, x_dev, x_test = map(build_inputs, [idx_train, idx_dev, idx_test])
y_train, y_dev, y_test = y[idx_train], y[idx_dev], y[idx_test]

model = build_model(atom_vocab_size, bond_vocab_size)

history = model.fit(
    x_train, y_train,
    validation_data=(x_dev, y_dev),
    epochs=1000,
    batch_size=32,
    callbacks=[
        EarlyStopping(patience=50, restore_best_weights=True),
        SelectiveVerboseCallback(total_epochs=1000)
    ],
    verbose=0
)

os.makedirs("models", exist_ok=True)
model.save("models/viscosity_final.keras")

print("Train R2:", r2_numpy(y_train, model.predict(x_train).flatten()))
print("Dev   R2:", r2_numpy(y_dev,   model.predict(x_dev).flatten()))
print("Test  R2:", r2_numpy(y_test,  model.predict(x_test).flatten()))


I0000 00:00:1765700526.283727  962109 service.cc:145] XLA service 0x77e27000bf40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765700526.283795  962109 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
I0000 00:00:1765700549.015312  962109 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1 - loss: 97.266747, val_loss: 12.758076
Epoch 2 - loss: 11.275120, val_loss: 14.831838
Epoch 3 - loss: 7.950568, val_loss: 15.828735
Epoch 4 - loss: 3.897224, val_loss: 0.671229
Epoch 5 - loss: 3.212166, val_loss: 3.792301
Epoch 50 - loss: 0.122934, val_loss: 0.153441
Epoch 100 - loss: 0.090418, val_loss: 0.104600
Epoch 150 - loss: 0.065704, val_loss: 0.071188
Epoch 200 - loss: 0.055696, val_loss: 0.081959
168/168 ━━━━━━━━━━━━━━━━━━━━ 27s 56ms/step
Train R2: 0.9235160742399561
36/36 ━━━━━━━━━━━━━━━━━━━━ 15s 422ms/step
Dev   R2: 0.8889337277972653
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Test  R2: 0.8737756648689126


SyntaxError: invalid or missing encoding declaration for 'results/figure2_a_viscosity.png' (<string>)